# Text generation with deep learning

In [1]:
import sys
import os
import os.path
import random
import collections
import shutil
import time
import glob
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from math import pi
import scipy
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import torchvision
from torchvision import transforms, datasets
from torch.nn.utils.rnn import pad_sequence


%matplotlib inline

from torch.utils.data import DataLoader

# для визуализации
# from bokeh.io import show,output_notebook
# from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper, BasicTicker, PrintfTickFormatter,ColorBar
# from bokeh.models import FuncTickFormatter
# from bokeh.plotting import figure

# output_notebook()

C:\Users\User\anaconda3\envs\gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Задача**

   Обучить character recurrent neural network (char RNN) на датасете статей arxiv, а также на своих данных и исследовать её свойства.

Определим структуру сети, в качестве основного блока используем **GRU**

In [126]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
#         print("input before encoding: ", input)
#         print("input shape before encoding: ", input.shape)
#         print("================================================")
        input = self.encoder(input.view(1, -1))
#         print("input after encoding: ", input)
#         print("input shape after encoding: ", input.shape)
#         print("================================================")
#         print("input.view(-1): ", input.view(-1))
#         print("================================================")
#         print("input.view(-1) size: ", len(input.view(-1)))
#         print("================================================")
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
#         print("gru passed")
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size)
    
    def save_model(model, filename='rnn.ckpt'):
        checkpoint = {'input_size': model.input_size,
                      'hidden_size': model.hidden_size,
                      'output_size': model.output_size,
                      'n_layers': model.n_layers,
                      'state_dict': model.state_dict()}
        with open(filename, 'wb') as f:
            torch.save(checkpoint, f)

    def load_model(filename):

        with open(filename, 'rb') as f:
            checkpoint = torch.load(f)

        model = RNN(input_size = checkpoint['input_size'], output_size = checkpoint['output_size'], hidden_size=checkpoint['hidden_size'], n_layers=checkpoint['n_layers'])
        model.load_state_dict(checkpoint['state_dict'])

        return model

In [124]:
torch.zeros(2, 1, 10)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

# Пример реализации класса для получения данных из датасета

In [4]:
class TextDataset(Dataset):
#     def __init__(self, text, chunk_len=10, chunks_number=100):
    def __init__(self, text, chunk_len=10, chunks_number=100):
        self.chunks_number = chunks_number # количество блоков
        self.chunk_len = chunk_len  # длина одного блока (на сколько букв смотрим назад при предсказании)
        self.text = text
        self.all_symbols = list({x for x in self.text})
    
    def __len__(self):
        return self.chunks_number
    
    def _get_tensor(self, text):   # ставит в соответствие каждой букве из текста её индекс
        return torch.LongTensor(list(map(self.all_symbols.index, text))).cuda()
    
    def _decode_vector(self, text):   # декодирует вектор из цифр (text) в текст
        text = text.view(-1).cpu().numpy()
        if text.shape[0] == 1:
            text = list(text)
        return ''.join([self.all_symbols[x] for x in text])
    
    def _get_random_slice(self):  # берет случайный срез букв из текста
        start_index = random.randint(0, len(self.text) - self.chunk_len)
        end_index = start_index + self.chunk_len + 1
        return self.text[start_index:end_index]
    
    # 1 саммари - 100 инпутов и 100 таргетов 
    def __getitem__(self, idx):
        chunk = self._get_random_slice()  
        # self._get_tensor(chunk[:-1]) - текущее состояние, self._get_tensor(chunk[1:]) - следующее состояние (target)??
        return self._get_tensor(chunk[:-1]), self._get_tensor(chunk[1:])  # chunk[:-1] не включает последний эл-т, chunk[1:] не включает первый эл-т

# Здесь должен быть код для считывания данных и создания даталоадера

In [270]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# arxiv_dataset = pd.read_csv('/content/drive/MyDrive/Универ/arxiv.csv') 
arxiv_dataset = pd.read_csv('arxiv.csv') 
arxiv_dataset

,link,time,favorites,rts,authors,category,published,summary,title,tweeted
0,arxiv.org/abs/1611.10003,NaN,NaN,NaN,"[Tom A. F. Anderson, C. -H. Ruan]",q-bio.NC,2016-11-30 05:17:11,In summary of the research findings presented ...,Vocabulary and the Brain: Evidence from Neuroi...,0
1,arxiv.org/abs/1611.10007,NaN,NaN,NaN,"[M. Amin Rahimian, Amir G. Aghdam]",cs.SY,2016-11-30 05:37:11,"In this paper, structural controllability of a...",Structural Controllability of Multi-Agent Netw...,0
2,arxiv.org/abs/1611.10010,NaN,NaN,NaN,"[Debidatta Dwibedi, Tomasz Malisiewicz, Vijay ...",cs.CV,2016-11-30 06:00:47,We present a Deep Cuboid Detector which takes ...,Deep Cuboid Detection: Beyond 2D Bounding Boxes,0
3,arxiv.org/abs/1611.10012,2016-12-01 01:46:12,11.0,2.0,"[Jonathan Huang, Vivek Rathod, Chen Sun, Mengl...",cs.CV,2016-11-30 06:06:15,"In this paper, we study the trade-off between ...",Speed/accuracy trade-offs for modern convoluti...,1
4,arxiv.org/abs/1611.10014,NaN,NaN,NaN,"[Yoones Hashemi, Amir H. Banihashemi]",cs.IT,2016-11-30 06:12:45,"In this paper, we propose a characterization o...",Characterization and Efficient Exhaustive Sear...,0
...,...,...,...,...,...,...,...,...,...,...
27183,arxiv.org/abs/1710.04176,NaN,NaN,NaN,"[C. -C. Jay Kuo, Yueru Chen]",cs.CV,2017-10-11 17:08:28,Being motivated by the multilayer RECOS (REcti...,On Data-Driven Saak Transform,0
27184,arxiv.org/abs/1710.04177,NaN,NaN,NaN,"[Heather Mattie, Kenth Engø-Monsen, Rich Ling,...",cs.SI,2017-10-11 17:12:05,"Understanding tie strength in social networks,...",The Social Bow Tie,0
27185,arxiv.org/abs/1710.04196,NaN,NaN,NaN,"[Robin Scheibler, Eric Bezzam, Ivan Dokmanić]",cs.MS,2017-10-11 17:44:41,"We present pyroomacoustics, a software package...",Pyroomacoustics: A Python package for audio ro...,0
27186,arxiv.org/abs/1710.04200,NaN,NaN,NaN,"[Yijun Li, Jia-Bin Huang, Narendra Ahuja, Ming...",cs.CV,2017-10-11 17:56:59,Joint image filters leverage the guidance imag...,Joint Image Filtering with Deep Convolutional ...,0


In [283]:
# torch.zeros(2, 3, 4)[0]

In [156]:
def collate_fn(input_data):  # на вход приходит arxiv_dataset['summary']
    inputs = []
    targets = []
    for item in tqdm(input_data):
        item_ = TextDataset(item)
#         for chunks_n in range(item_.chunks_number):
#             inputs.append(item_.__getitem__(chunks_n)[0])
#             targets.append(item_.__getitem__(chunks_n)[1])
        inputs.append(item_.__getitem__(0)[0])
        targets.append(item_.__getitem__(0)[1])
#     print("collate_fn: before pad", inputs)
    inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
#     print("collate_fn: after pad", inputs)
    targets = pad_sequence(targets, batch_first=True, padding_value=0)
    return {'inputs': inputs, 'targets': targets}
    # return inputs, targets

In [153]:
collate_fn(arxiv_dataset['summary'])


 81%|███████████████████████████████████████████████████████████▊              | 21968/27188 [00:13<00:03, 1598.92it/s]


 83%|█████████████████████████████████████████████████████████████             | 22449/27188 [00:14<00:03, 1540.46it/s]Exception ignored in: <function tqdm.__del__ at 0x0000024CC398E550>
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\envs\gpu\lib\site-packages\tqdm\std.py", line 1162, in __del__
    self.close()
  File "C:\Users\User\anaconda3\envs\gpu\lib\site-packages\tqdm\std.py", line 1291, in close
    if self.last_print_t < self.start_t + self.delay:
AttributeError: 'tqdm' object has no attribute 'last_print_t'

100%|██████████████████████████████████████████████████████████████████████████| 27188/27188 [00:16<00:00, 1601.30it/s]


collate_fn: after pad tensor([[ 5,  2, 12,  ..., 23, 26, 18],
        [ 1, 15, 31,  ..., 26, 21, 28],
        [ 5, 37, 40,  ..., 30, 23,  5],
        ...,
        [ 1, 26, 27,  ..., 16,  7,  8],
        [31, 13, 21,  ...,  5, 30, 38],
        [20, 15, 32,  ...,  6, 12,  6]], device='cuda:0')


{'inputs': tensor([[ 5,  2, 12,  ..., 23, 26, 18],
         [ 1, 15, 31,  ..., 26, 21, 28],
         [ 5, 37, 40,  ..., 30, 23,  5],
         ...,
         [ 1, 26, 27,  ..., 16,  7,  8],
         [31, 13, 21,  ...,  5, 30, 38],
         [20, 15, 32,  ...,  6, 12,  6]], device='cuda:0'),
 'targets': tensor([[23, 29, 15,  ...,  3,  5, 25],
         [28, 15,  1,  ..., 27,  4,  2],
         [43, 16,  5,  ..., 38, 15, 43],
         ...,
         [ 1, 23, 32,  ..., 33,  1, 26],
         [31, 30,  1,  ...,  5, 23, 13],
         [30, 15, 36,  ...,  6, 27, 22]], device='cuda:0')}

In [110]:
# dict_ = collate_fn(arxiv_dataset['summary'])

In [111]:
# dict_['inputs'][0]

In [112]:
# len(dict_['targets'])

In [157]:
from torch.utils.data import DataLoader
batch_size = 1
loader = DataLoader(arxiv_dataset['summary'], 
                            batch_size, 
                            shuffle=True,
#                             num_workers=2,
                            collate_fn=collate_fn,
#                             drop_last = True,
                            )

In [289]:
# for inputs, targets in loader:
#   print(inputs[0].size(-1))
# for i, batch in enumerate(loader):
#   print(batch['inputs'].size(-1))

Смотрим, сколько уникальных символов есть в датасете - это и будет размер выходного вектора у нейронной сети

In [114]:
unique_symbols = []
for i in range(len(arxiv_dataset['summary'])):
    Text_Dataset = TextDataset(arxiv_dataset['summary'][i])
    for symbol in Text_Dataset.all_symbols:
        if symbol not in unique_symbols:
            unique_symbols.append(symbol)

In [154]:
len(unique_symbols)

97

In [292]:
# Text_Dataset = TextDataset(arxiv_dataset['summary'][5818])
# len(Text_Dataset.all_symbols)

In [160]:
# unique_symbols = len(Text_Dataset.all_symbols)
n_layers = 2
hidden_size = 1000

In [ ]:
# torch.zeros(2, 1, 100)

In [295]:
# unique_symbols

In [116]:
device = torch.device("cuda")

In [161]:
net = RNN(len(unique_symbols), hidden_size, len(unique_symbols), n_layers)

In [162]:
net = net.to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss().cuda()

# Здесь нужно реализовать функцию для генерации текста с помощью сети

In [299]:
def generate(net, device, dataset, prime_str='А', predict_len=100, temperature=0.8):
    #Code to generate string based on input sequence (prime_str)
    
    # predict = model(batch['data'].to(device))
    #     loss = loss_func(predict.view(-1, n_classes),

    predicted_string = net(prime_str)

    return predicted_string

In [300]:
# generate(net, device, arxiv_dataset['summary'])

# Здесь должен быть цикл тренировки

In [14]:
epochs = 100

In [163]:
start = time.time()

net.train()
# hidden = net.init_hidden()
for epoch in range(1, epochs+1):
#     hidden = net.init_hidden().to(device)
#     hidden = torch.zeros(n_layers, 1, 1000)
    # for inputs, targets in loader:
    for i, batch in enumerate(loader):
        #Your code for training here
        # hidden = tuple(([Variable(var.data) for var in hidden]))
        optimizer.zero_grad()
        
#         print(batch['inputs'])
#         print(hidden.to(device).size(-1))
        output, hidden = net(batch['inputs'].to(device), hidden)
#         loss = criterion(output, batch['targets'].to(device))
#         loss.backward()
        optimizer.step()

print("Saving...")
RNN.save_model(net, "TrainedModel.ckpt")

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 995.33it/s]


RuntimeError: input.size(-1) must be equal to input_size. Expected 1000, got 10000

In [ ]:
print(len(hidden[0][0]))

In [ ]:
net(inputs[0], hidden)[0].shape

In [ ]:
targets[0]

In [ ]:
plt.figure()
plt.plot(all_losses)

## Результаты тренировки

In [ ]:
print(generate(net, device, arxiv_dataset, 'In this research', 200 ), '\n')

# Здесь должны быть эксперименты с архитектурой и параметрами сети

In [2]:
device = torch.device("cuda")

In [3]:
arxiv_dataset = pd.read_csv('arxiv.csv') 
arxiv_dataset

,link,time,favorites,rts,authors,category,published,summary,title,tweeted
0,arxiv.org/abs/1611.10003,NaN,NaN,NaN,"[Tom A. F. Anderson, C. -H. Ruan]",q-bio.NC,2016-11-30 05:17:11,In summary of the research findings presented ...,Vocabulary and the Brain: Evidence from Neuroi...,0
1,arxiv.org/abs/1611.10007,NaN,NaN,NaN,"[M. Amin Rahimian, Amir G. Aghdam]",cs.SY,2016-11-30 05:37:11,"In this paper, structural controllability of a...",Structural Controllability of Multi-Agent Netw...,0
2,arxiv.org/abs/1611.10010,NaN,NaN,NaN,"[Debidatta Dwibedi, Tomasz Malisiewicz, Vijay ...",cs.CV,2016-11-30 06:00:47,We present a Deep Cuboid Detector which takes ...,Deep Cuboid Detection: Beyond 2D Bounding Boxes,0
3,arxiv.org/abs/1611.10012,2016-12-01 01:46:12,11.0,2.0,"[Jonathan Huang, Vivek Rathod, Chen Sun, Mengl...",cs.CV,2016-11-30 06:06:15,"In this paper, we study the trade-off between ...",Speed/accuracy trade-offs for modern convoluti...,1
4,arxiv.org/abs/1611.10014,NaN,NaN,NaN,"[Yoones Hashemi, Amir H. Banihashemi]",cs.IT,2016-11-30 06:12:45,"In this paper, we propose a characterization o...",Characterization and Efficient Exhaustive Sear...,0
...,...,...,...,...,...,...,...,...,...,...
27183,arxiv.org/abs/1710.04176,NaN,NaN,NaN,"[C. -C. Jay Kuo, Yueru Chen]",cs.CV,2017-10-11 17:08:28,Being motivated by the multilayer RECOS (REcti...,On Data-Driven Saak Transform,0
27184,arxiv.org/abs/1710.04177,NaN,NaN,NaN,"[Heather Mattie, Kenth Engø-Monsen, Rich Ling,...",cs.SI,2017-10-11 17:12:05,"Understanding tie strength in social networks,...",The Social Bow Tie,0
27185,arxiv.org/abs/1710.04196,NaN,NaN,NaN,"[Robin Scheibler, Eric Bezzam, Ivan Dokmanić]",cs.MS,2017-10-11 17:44:41,"We present pyroomacoustics, a software package...",Pyroomacoustics: A Python package for audio ro...,0
27186,arxiv.org/abs/1710.04200,NaN,NaN,NaN,"[Yijun Li, Jia-Bin Huang, Narendra Ahuja, Ming...",cs.CV,2017-10-11 17:56:59,Joint image filters leverage the guidance imag...,Joint Image Filtering with Deep Convolutional ...,0


In [8]:
class TextDataset(Dataset):
    def __init__(self, text, chunk_len=10, chunks_number=100):
        self.chunks_number = chunks_number # количество блоков
        self.chunk_len = chunk_len  # длина одного блока (на сколько букв смотрим назад при предсказании)
        self.text = text
        self.all_symbols = list({x for x in self.text})
    
    def __len__(self): # возвращает число чанков (случайных слайсов по тексту?)
        return self.chunks_number
    
    def _get_tensor(self, text):   # ставит в соответствие каждой букве из текста её индекс
        return torch.LongTensor(list(map(self.all_symbols.index, text))).cuda()
    
    def _decode_vector(self, text):   # декодирует вектор из цифр (text) в текст
        text = text.view(-1).cpu().numpy()
        if text.shape[0] == 1:
            text = list(text)
        return ''.join([self.all_symbols[x] for x in text])
    
    def _get_random_slice(self):  # берет случайный срез букв из текста
        start_index = random.randint(0, len(self.text) - self.chunk_len)
        end_index = start_index + self.chunk_len + 1
#         print("start: ", start_index, "end: ", end_index)
        return self.text[start_index:end_index]
    
    # 1 саммари - 100 инпутов и 100 таргетов 
    def __getitem__(self, idx):
#         print("==========================================")
        chunk = self._get_random_slice()
#         print("text:", self.text)
#         print("chunk:", chunk)
#         print("symbols:")
#         for symb in self.all_symbols:
#             print(self._get_tensor(symb), symb)
        # self._get_tensor(chunk[:-1]) - текущее состояние, self._get_tensor(chunk[1:]) - следующее состояние (target)??
        if len(chunk) == self.chunk_len + 1: # стандартное поведение
#             print("check passed, return:", self._get_tensor(chunk[:-1]), self._get_tensor(chunk[1:]))
            return self._get_tensor(chunk[:-1]), self._get_tensor(chunk[1:])  # chunk[:-1] не включает последний эл-т, chunk[1:] не включает первый эл-т
        else: # попали на последний символ
            print("check unpassed, return:", torch.LongTensor([self.all_symbols.index(char) for char in chunk]).cuda(), torch.LongTensor([self.all_symbols.index(char) for char in chunk[1:]] + [97]).cuda())
            return torch.LongTensor([self.all_symbols.index(char) for char in chunk]).cuda(), torch.LongTensor([self.all_symbols.index(char) for char in chunk[1:]] + [97]).cuda()
#             return torch.LongTensor([str.all_symbols.index(char) for char in text] + [99]).cuda()

In [5]:
symbols = []
for i in range(len(arxiv_dataset['summary'])):
    Text_Dataset = TextDataset(arxiv_dataset['summary'][i])
    for symbol in Text_Dataset.all_symbols:
        if symbol not in symbols:
            symbols.append(symbol)

In [7]:
# params
unique_symbols = len(symbols) + 1
n_layers = 2
hidden_size = 100
epochs = 100
batch_size = 1

In [9]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        # input_size (==unique_symbols) – size of the dictionary of embeddings
        # hidden_size – the size of each embedding vector
        # Вход: (*) IntTensor или LongTensor произвольной формы, содержащие индексы для извлечения
        # Output: (∗, H), where * is the input shape and H = hidden_size
        
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, inputs, hidden):
#         print("input before encoding: ", inputs)
#         print("input shape before encoding: ", input.shape)
#         print("================================================")
        inputs = self.encoder(inputs.view(1, -1))
        # Embedding — кажому элементу из input-а ставит некоторое векторное представление размера hidden_size
        # Например, если пришёл один тензор из 10 элементов (10 закодированных символов char), то кажому из этих 10 элементов будет поставлен в соответствии вектор размера hidden_size 
        # вернёт тензор второй вложенности размера len(inputs) * hidden_size
#         print("input after encoding: ", inputs)
#         print("input shape after encoding: ", input.shape)
#         print("================================================")
#         print("input.view(-1): ", input.view(-1))
#         print("================================================")
#         print("input.view(-1) size: ", len(input.view(-1)))
#         print("================================================")
        output, hidden = self.gru(inputs.view(1, 1, -1), hidden)
#         print("gru passed")
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size)
    
    def save_model(model, filename='rnn.ckpt'):
        checkpoint = {'input_size': model.input_size,
                      'hidden_size': model.hidden_size,
                      'output_size': model.output_size,
                      'n_layers': model.n_layers,
                      'state_dict': model.state_dict()}
        with open(filename, 'wb') as f:
            torch.save(checkpoint, f)

    def load_model(filename):

        with open(filename, 'rb') as f:
            checkpoint = torch.load(f)

        model = RNN(input_size = checkpoint['input_size'], output_size = checkpoint['output_size'], hidden_size=checkpoint['hidden_size'], n_layers=checkpoint['n_layers'])
        model.load_state_dict(checkpoint['state_dict'])

        return model

In [10]:
net = RNN(unique_symbols, hidden_size, unique_symbols, n_layers)

net = net.to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss().cuda()

In [11]:
def collate_fn(input_data):  # на вход приходит arxiv_dataset['summary']
    inputs = []
    targets = []
    for item in input_data:
        item_ = TextDataset(item, chunk_len=1).__getitem__(0)
#         for chunks_n in range(item_.chunks_number):
#             inputs.append(item_.__getitem__(chunks_n)[0])
#             targets.append(item_.__getitem__(chunks_n)[1])
        inputs.append(item_[0])
        targets.append(item_[1])
#     print("collate_fn before pad:", inputs, targets)

    inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
    targets = pad_sequence(targets, batch_first=True, padding_value=0)
    
    # pad_sequence - возвращает Tensor размерности T x B x * or B x T x * (если batch_first=True)
    # B это batch size. (Он равен количеству элементов в sequence)
    # T длина наибольшей sequence. 
    # L длина sequence. 
    #* любое кол-во конечных измерений, включая none.
    
#     print("collate_fn after pad:", inputs, targets)
    return {'inputs': inputs, 'targets': targets}
    # return inputs, targets

In [12]:
loader = DataLoader(arxiv_dataset['summary'], 
                            batch_size, # как много sample загружать за 1 батч
#                             shuffle=True, # перетасовка данных между эпохами
#                             num_workers=2,
                            collate_fn=collate_fn, # функция-обработчик датасета (объединяет список сэмплов, чтобы сформировать мини-партию тензоров)
#                             drop_last = True, # удалять ли последний незавершённый batch (если размер датасета не делится на batch_size)
                            )

In [13]:
start = time.time()

net.train()
# hidden = net.init_hidden()
for epoch in range(1, epochs+1):
    hidden = net.init_hidden().to(device)
#     hidden = torch.zeros(n_layers, 1, 1000)
    # for inputs, targets in loader:
    for i, batch in enumerate(tqdm(loader)):
#         print("i:", i)
        hidden = hidden.detach()
        # hidden = tuple(([Variable(var.data) for var in hidden]))
        optimizer.zero_grad()
        
#         print(batch['inputs'])
#         print(hidden.to(device).size(-1))
        output, hidden = net(batch['inputs'].to(device), hidden)
        loss = criterion(output, torch.flatten(batch['targets']).to(device))
        loss.backward()
        optimizer.step()

print("Saving...")
RNN.save_model(net, "TrainedModel.ckpt")

  2%|█▎                                                                           | 470/27188 [00:02<02:27, 181.20it/s]

check unpassed, return: tensor([19], device='cuda:0') tensor([666], device='cuda:0')


  5%|████                                                                        | 1470/27188 [00:08<02:17, 187.32it/s]

check unpassed, return: tensor([28], device='cuda:0') tensor([666], device='cuda:0')


  6%|████▋                                                                       | 1680/27188 [00:09<02:22, 179.26it/s]

...

 70%|█████████████████████████████████████████████████████▏                      | 19026/27188 [04:49<01:30, 89.80it/s]

check unpassed, return: tensor([43], device='cuda:0') tensor([666], device='cuda:0')


 80%|████████████████████████████████████████████████████████████▎              | 21851/27188 [05:18<00:47, 111.75it/s]

check unpassed, return: tensor([17], device='cuda:0') tensor([666], device='cuda:0')


 84%|███████████████████████████████████████████████████████████████▊            | 22810/27188 [05:28<01:02, 69.50it/s]

KeyboardInterrupt



In [ ]:
str = TextDataset("We show that it is possible to use Bondy-Chvatal closure to design an FPT algorithm that decides whether or not it is possible to cover vertices of an input graph by at most k vertex disjoint paths in the complement of the input graph. More precisely, we show that if a graph has tree-width at most w and its complement is closed under Bondy-Chvatal closure, then it is possible to bound neighborhood diversity of the complement by a function of w only. A simpler proof where tree-depth is used instead of tree-width is also presented.", chunk_len=1)

In [172]:
str.all_symbols

['d',
 'M',
 's',
 'j',
 'v',
 'e',
 'r',
 'A',
 'u',
 'g',
 'n',
 'p',
 'T',
 ' ',
 'w',
 'c',
 'y',
 'l',
 'b',
 'i',
 '.',
 'W',
 'B',
 ',',
 'o',
 'P',
 't',
 'a',
 'F',
 'f',
 'h',
 'x',
 'C',
 'm',
 '-',
 'k',
 '']

In [137]:
chunk[:-1]

''

In [169]:
# str.all_symbols.append("")

In [170]:
# str._get_tensor(chunk[:-1])
str._get_tensor(chunk[:str.chunk_len])

tensor([20], device='cuda:0')

In [171]:
str._get_tensor(chunk[1:])

tensor([], device='cuda:0', dtype=torch.int64)

In [ ]:
return self._get_tensor(chunk[:-1]), self._get_tensor(chunk[1:])  # chunk[:-1] не включает последний эл-т, chunk[1:] не включает первый эл-т

In [133]:
chunk = str.text[534:536]
chunk

'.'

In [189]:
torch.LongTensor(list(map(str.all_symbols.index, '')))

tensor([], dtype=torch.int64)

In [212]:
text=''

In [213]:
str.all_symbols.index(text)

36

In [216]:
# torch.LongTensor([str.all_symbols.index(x) for x in text] + [len(str.all_symbols)])
torch.LongTensor([str.all_symbols.index(x) for x in text] + [99])

tensor([99])

In [208]:
torch.LongTensor(list(map(str.all_symbols.index, 'text')))

tensor([26,  5, 31, 26])

In [159]:
# symbols.append("")

In [160]:
# if "" in symbols:
#     print("YES")
# else:
#     print("NO")

YES


# Здесь будет работа с вашими датасетами (которые вы захотите использовать)